In [6]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from scipy.optimize import minimize
from collections import Counter

# === Load and normalize Iris data ===
X, y = load_iris(return_X_y=True)
X = X[y == 0][:50]  # Class: Setosa only
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# === Convert real data to target bitstring patterns (2 bits per feature) ===
def feature_to_bitpair(val):
    if val <= 0.16: return "00"
    elif val <= 0.5: return "01"
    elif val <= 0.83: return "10"
    else: return "11"

def features_to_bitstring(x):
    return ''.join([feature_to_bitpair(f) for f in x])

bitstring_target = [features_to_bitstring(x) for x in X_scaled]

# === Build probability distribution for training (target)
def get_target_distribution(bits_list):
    counter = Counter(bits_list)
    total = sum(counter.values())
    probs = np.zeros(256)  # 2^8
    for b, c in counter.items():
        idx = int(b, 2)
        probs[idx] = c / total
    return probs

target_probs = get_target_distribution(bitstring_target)

# === VQC circuit with 8 qubits
n_qubits = 8
param_size = 16  # 2 layers × 8 qubits
shots = 1024

def create_vqc(params):
    qc = QuantumCircuit(n_qubits)
    for i in range(n_qubits):
        qc.ry(params[i], i)
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)
    for i in range(n_qubits):
        qc.rz(params[i + 8], i)
    qc.measure_all()
    return qc

# === Evaluate distribution from circuit
def get_probs_from_params(params):
    qc = create_vqc(params)
    backend = Aer.get_backend("qasm_simulator")
    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=shots)
    counts = job.result().get_counts()
    
    probs = np.zeros(256)
    for bit, count in counts.items():
        idx = int(bit[::-1], 2)
        probs[idx] = count / shots
    return probs

# === Loss function: MSE between target vs generated distribution
def loss(params):
    gen_probs = get_probs_from_params(params)
    return np.mean((gen_probs - target_probs) ** 2)

# === Train parameters using classical optimizer
initial_theta = np.random.uniform(0, 2 * np.pi, param_size)
print("Training VQC generator... (takes 1–3 min)")
result = minimize(loss, initial_theta, method="COBYLA", options={"maxiter": 100})
trained_theta = result.x

# === Sampling from trained circuit
def generate_samples(params, n=300):
    probs = get_probs_from_params(params)
    samples = []
    for i, p in enumerate(probs):
        bit = format(i, "08b")
        samples += [bit] * int(p * n)
    return samples[:n]

generated_bitstrings = generate_samples(trained_theta, n=5000)

# === Decode bitstring → scaled feature
def decode_bitpair(pair):
    return {
        "00": 0.0,
        "01": 0.33,
        "10": 0.66,
        "11": 1.0
    }.get(pair, 0.0)

def decode_bitstring_to_features(bs):
    return [decode_bitpair(bs[i:i+2]) for i in range(0, len(bs), 2)]

decoded_features = np.array([decode_bitstring_to_features(bs) for bs in generated_bitstrings])

# === Inverse transform to get Iris-like data
generated_iris = scaler.inverse_transform(decoded_features)

df = pd.DataFrame(generated_iris, columns=["sepal length", "sepal width", "petal length", "petal width"])
df["label"] = "Generated (VQC)"

# === แสดงผล
print("\n📊 ตัวอย่างข้อมูลที่สร้างจาก VQC:")
print(df.head())

# === Save to CSV
df.to_csv("generated_iris_8qubit.csv", index=False)


Training VQC generator... (takes 1–3 min)

📊 ตัวอย่างข้อมูลที่สร้างจาก VQC:
   sepal length  sepal width  petal length  petal width            label
0           4.3          2.3           1.0         0.10  Generated (VQC)
1           4.3          2.3           1.0         0.10  Generated (VQC)
2           4.3          2.3           1.0         0.10  Generated (VQC)
3           4.3          2.3           1.0         0.10  Generated (VQC)
4           4.3          2.3           1.0         0.43  Generated (VQC)
